## Q1

__Kernel SVM__

From  sklearn  import  the  Breast  Cancer  dataset.   Draw  the  contour plots generated by fitting an SVM with the following kernels to the data (you can use sklearn’s implementation of SVM):

- (a)  Linear kernel
- (b)  Radial Basis Function (RBF) kernel
- (c)  Polynomial kernel of degree 5



#### load breast cancer data from sklearn

In [25]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC

cancer = load_breast_cancer()
X = cancer.data[:, :2]
labels = cancer.target

In [26]:
def plot_contour(x, 
                 y, 
                 labels,
                 models,
                 row,
                 col,
                 figsize,
                 cmap1,
                 cmap2,
                 titles,
                 multiple=True):
    # meshgrid
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    
    fig, sub = plt.subplots(row,
                            col, 
                            figsize=figsize)
    try:
        s = sub.flatten()
    except Exception as e:
        print(e)
        s = sub
    for model, title, ax in zip(models, titles, s):
        Xy = model.predict(np.c_[xx.ravel(), yy.ravel()])
        Xy = Xy.reshape(xx.shape)
        ax.contourf(xx, yy, Xy, cmap=cmap1)
        ax.scatter(x, y, c=labels, s=50, cmap=cmap2)
        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_title(title)
    plt.show()

In [ ]:
FIGSIZE = (20,6)
models = (SVC(kernel='linear'),
          SVC(kernel='rbf'),
          SVC(kernel='poly', degree=5)
         )
models = (clf.fit(X, labels) for clf in models)

TITLES = ('SVC with linear kernel',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 5) kernel')

plot_contour(x=X[:, 0],
             y=X[:, 1],
             labels=labels,
             models=models,
             row=1,
             col=3,
             figsize=FIGSIZE,
             cmap1=plt.cm.coolwarm,
             cmap2=plt.cm.coolwarm,
             titles=TITLES)

## Q2

__Kernel logistic regression__  

In this exercise,  we're interested in using RBF kernel in  logistic  regression.   Report  the  average  area  under  curve  (AUC)  of  10-fold  cross-validation  (where  in  each  fold  you  use  20%  of  the  data  for  testing)  on  the  Breast Cancer dataset using RBF logistic regression (you can use sklearn implementations).

In [ ]:
X = cancer.data
labels = cancer.target

def rbf_kernel(x, mean, sd):
    return np.exp(-1/(2*sd**2)*(x-mean)**2)


def evaluate(X_train, X_test, y_train, y_test, fold):
    clf = LogisticRegression(solver='lbfgs')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=y_pred)
    auc_value = auc(fpr, tpr)
    plot_roc(fpr, tpr, auc_value, fold=fold)

def plot_roc(fpr, tpr, auc_value, fold=None):
    plt.subplot(2,5,fold)
    plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve (area = %0.3f)' % auc_value)
    plt.plot([0, 1], [0, 1], lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    if fold != None:
        plt.title('ROC plot of fold {}\n auc = {}'.format(fold, auc_value))
    else:
        plt.title('ROC plot, auc = {}'.format(auc_value))
    plt.legend(loc="lower right")
    plt.grid(True)

def k_fold_evaluate(X, y):
    # initialization
    accuracy = []
    precision = []
    recall=[]
    f1 = []
    fold = 1
    skf = StratifiedKFold(n_splits=10, random_state=100, shuffle= True)
    
    # rbf
    X = rbf_kernel(X, 0, 1)
    for train_index, test_index in skf.split(X, y):
        print(len(test_index)/(len(X)))
        X_train, X_test, y_train, y_test = X[train_index, :], X[test_index, :], y[train_index], y[test_index]
        evaluate(X_train, X_test, y_train, y_test, fold)
        fold += 1
        
plt.figure(figsize=(25, 8))     
k_fold_evaluate(X, labels)
plt.show()

## Q3

__Soft-margin kernel SVM__

Implement  soft-margin SVM with a linear kernel, optimize the objective function according to slides #119-120 of the lecture slides on ”Lectures 15-16 Maximum Margin Linear Classifiers” onthe course website.1Fit your implemented classifier to the Breast Cancer dataset anddraw the contour plot similar to question 1.

$$J(w, b) = \frac{1}{2}w^TXw + C\sum^m_{i=1} \max(0, 1-y_i (X_i \cdot w + b)) $$

__Derivative:__

$$\frac{\partial J}{\partial w} = C \sum^m_{i=1} \mathbb{I} (y_i (X_i \cdot w +b)<1) \cdot (-y_i X_i) + X_i w$$


$$\frac{\partial J}{\partial b} = C \sum_{i=1}^m \mathbb{I} (y_i (X_i \cdot w +b)<1) \cdot (-y_i)$$

__Update:__

$$w \leftarrow w - \eta \frac{\partial J}{\partial w}$$

$$b \leftarrow b - \eta \frac{\partial J}{\partial b}$$


#### Augmented matrix $[w, b]$:

$$J(w, b) = \frac{1}{2}w^TXw + C\sum^{m}_{i=1} \max(0, 1-y_i (X_i \cdot w )) $$


$$\frac{\partial J}{\partial w} = C \sum^{m+1}_{i=1} \mathbb{I} (y_i (X_i \cdot w )<1) \cdot (-y_i X_i) + X_i w$$

In [ ]:
import numpy as np

class SoftMarginSVM:
    def __init__(self,
                 X,
                 y,
                 C=3,
                 learning_rate=0.001,
                 n_iter=100000):
        # augmented matrix X = [w, b]
        self.X = np.hstack([X, np.ones([X.shape[0],1])])
        self.y = np.where(y==0, -1, 1) # original y is (0,1), otherwise 
        self.n_sample=self.X.shape[0]
        self.n_features=self.X.shape[1]
        self.W=np.random.random(self.n_features)-0.5
        self.C=C
        self.learning_rate=learning_rate
        self.n_iter=n_iter
        
    def loss_func(self):
        hinge_loss = np.maximum(0,1-self.y*np.dot(self.X,self.W))
        slack_term = hinge_loss.mean()*self.C
        w = self.W[:-1]
        cost_f = 0.5*np.sum(w**2)
        return cost_f + slack_term

    def d_obj_func(self):
        I=np.where(self.y*np.dot(self.X,self.W)>=1, 0, -1)[:,None]
        dJ=np.zeros(self.W.shape)
        dJ[:-1]=(I*self.X[:,:-1]*(np.array(self.y)[:,np.newaxis])).mean(axis=0)*self.C+self.W[:-1] 
        dJ[-1]=(I*self.C*(np.array(self.y)[:,np.newaxis])).mean(axis=0)                     
        return dJ

    def train(self):
        loss=[]
        for iter_ in range(self.n_iter):
            dJ=self.d_obj_func()
            self.W = self.W - self.learning_rate*dJ
            loss_per_iter = self.loss_func()
            loss.append(loss_per_iter)
            if iter_%10000 == 0:
                print('dj:', dJ)
                print('W =', self.W)
                print('loss at iter {}:'.format(iter_), loss_per_iter)
        self.losses=loss


    def predict(self,x):
        # augmented matrix X = [w, b]
        x = np.hstack([x,np.ones([x.shape[0],1])])
        y_preds=[]
        for i in x:
            y_pred=1 if np.sign(i.dot(self.W))>=0 else -1
            y_preds.append(y_pred)
        return np.array(y_preds)

In [ ]:
softsvm_gd = SoftMarginSVM(X,labels)
softsvm_gd.train()

svm_sklearn=SVC(C=3,kernel='linear')
svm_sklearn.fit(X,labels)

print('not sklearn svm weights:', softsvm_gd.W)
print('sklearn svm weights:', svm_sklearn.coef_, 
      'sklearn svm weights:', svm_sklearn.intercept_)

In [ ]:
## plot result
plot_contour(x=X[:, 0],
             y=X[:, 1],
             labels=labels,
             models=(
                     softsvm_gd,
                     svm_sklearn
                    ),
             row=1,
             col=2,
             figsize=FIGSIZE,
             cmap1=plt.cm.coolwarm, 
             cmap2=plt.cm.coolwarm,
             titles=(
                    'soft margin svm (not sklearn)',
                     'svm sklearn'
                     ))